In [46]:
# Imports
import random
import scipy
import scipy.io
import sklearn.metrics
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
% matplotlib inline

In [263]:
def train_classifier(samples, labels):
    training_samples = samples[labels == 1]
    s_mean_clamped = np.squeeze(np.asarray(np.mean(training_samples, 0)))
    s_cov_clamped = np.cov(training_samples.T)
    m = s_mean_clamped
    C = s_cov_clamped - np.dot(np.asmatrix(s_mean_clamped).T, np.asmatrix(s_mean_clamped))
    delta = np.zeros(s_cov_clamped.shape)
    np.fill_diagonal(delta, 1. / (1. - np.multiply(m, m)))
    w = delta - np.linalg.inv(C)
    theta = np.arctanh(m) - np.dot(w, m)
    F = -0.5 * np.dot(np.dot(m, w), m) - np.dot(theta, m) + 0.5 * np.dot(1 + m, np.log(0.5 * (1 + m))) + 0.5 * np.dot(1 - m, np.log(0.5 * (1 - m)))
    Z = np.exp(-F)
    return w, theta, Z

def calculate_probabilities(samples, w, theta, normalizing_constant):
    return np.exp(-calculate_energy(samples, w, theta)) / normalizing_constant

def calculate_energy(samples, w, theta):
    f = np.dot(samples, np.dot(w, samples.T))
    # Also allow samples consisting of one sample (an array, so f.ndim == 1)
    # Therefore, only take the diagonal in the two dimensional case
    if f.ndim == 2:
        f = np.diagonal(f)
    return np.squeeze(np.asarray(-0.5 * f - np.dot(theta.T, samples.T)))

def generate_samples(s, w, theta, num_burn_in=50, num_samples=500, show_transition_probabilities=False):
    num_neurons = w.shape[0]
    
    # Initialize the matrix of generated samples
    X = np.empty((0, num_neurons))
    
    # Iterate (first generate some samples during the burn-in period and then gather the samples)
    for iteration in range(num_samples):
        for burn_in in range(num_burn_in + 1):
            # Store the original value of s
            s_original = s
            # Calculate the flip probabilities
            p_flip = 0.5 * (1 + np.tanh(np.multiply(-s, np.dot(w, s) + theta)))
            # Calculate transition probabilities
            p_transition = p_flip / float(num_neurons)
            p_stay = 1 - np.sum(p_transition)
            # Flip according to the probability distribution of flipping
            if random.random() <= 1 - p_stay:
                # Pick a random neuron
                neuron = random.randint(1, num_neurons) - 1
                if random.random() <= p_flip[neuron]:
                    s[neuron] *= -1
            # Add the state if the sample is not generated during the burn in period
            if burn_in >= num_burn_in:
                if show_transition_probabilities:
                    print('Transition probabilities for ', s_original,':', p_transition, ' (stay probability: ', p_stay, ')')
                X = np.vstack([X, s.T])
    return X

In [345]:
num_samples = 70
num_neurons = 50

X_train = np.random.binomial(1, 0.5, (num_samples, num_neurons)) * 2 - 1
y_train = (np.mean(X_train, axis=1) > 0) * 1

X_test = np.random.binomial(1, 0.5, (num_samples, num_neurons)) * 2 - 1
y_test = (np.mean(X_test, axis=1) > 0) * 1

In [346]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(alpha=1e-5, hidden_layer_sizes=(100, 10), random_state=1)
clf.fit(X_train, y_train)
predicted = clf.predict(X_test)
sklearn.metrics.accuracy_score(y_test, predicted)

0.77142857142857146

In [347]:
scores = []

for _ in range(10):
    w_pos, theta_pos, Z_pos = train_classifier(X_train, y_train)
    w_neg, theta_neg, Z_neg = train_classifier(X_train, 1 - y_train)

    num_new_samples = 10
    X = X_train
    y = y_train
    pos_samples = X_train[y_train == 1]
    neg_samples = X_train[y_train == 0]
    for _ in range(num_new_samples):
        pos_sample = np.asmatrix(pos_samples[np.random.randint(0, pos_samples.shape[0]), :]).T
        neg_sample = np.asmatrix(neg_samples[np.random.randint(0, neg_samples.shape[0]), :]).T
        sample_pos = generate_samples(pos_sample, w_pos, theta_pos.T, 0, 1)
        sample_neg = generate_samples(neg_sample, w_neg, theta_neg.T, 0, 1)
        X = np.vstack([X, sample_pos])
        y = np.hstack([y, 1])
        X = np.vstack([X, sample_neg])
        y = np.hstack([y, 0])

    clf = MLPClassifier(alpha=1e-5, hidden_layer_sizes=(100, 10), random_state=1)
    clf.fit(X, y)
    predicted = clf.predict(X_test)
    scores.append(sklearn.metrics.accuracy_score(y_test, predicted))
    
print(np.mean(scores), np.std(scores))

0.772857142857 0.0118666055185


In [407]:
num_samples = 50
num_neurons = 10

scores0 = []
scores1 = []
for _ in range(10):
    X_train = np.random.binomial(1, 0.5, (num_samples, num_neurons)) * 2 - 1
    y_train = (np.mean(X_train, axis=1) >= 0) * 1

    X_test = np.random.binomial(1, 0.5, (num_samples, num_neurons)) * 2 - 1
    y_test = (np.mean(X_test, axis=1) >= 0) * 1

    from sklearn.neural_network import MLPClassifier
    clf = MLPClassifier(alpha=1e-5, hidden_layer_sizes=(5, 5), random_state=1)
    clf.fit(X_train, y_train)
    predicted = clf.predict(X_test)
    scores0.append(sklearn.metrics.accuracy_score(y_test, predicted))

    noise = np.random.binomial(1, 0.2, X_train.shape) * -2 + 1
    X_train_noise = np.multiply(X_train, noise)
    w_pos, theta_pos, Z_pos = train_classifier(X_train_noise, y_train)
    w_neg, theta_neg, Z_neg = train_classifier(X_train_noise, 1 - y_train)

    num_new_samples = 10
    X = X_train
    y = y_train
    pos_samples = X_train[y_train == 1]
    neg_samples = X_train[y_train == 0]
    for _ in range(num_new_samples):
        pos_sample = np.asmatrix(pos_samples[np.random.randint(0, pos_samples.shape[0]), :]).T
        neg_sample = np.asmatrix(neg_samples[np.random.randint(0, neg_samples.shape[0]), :]).T
        sample_pos = generate_samples(pos_sample, w_pos, theta_pos.T, 500, 1)
        sample_neg = generate_samples(neg_sample, w_neg, theta_neg.T, 500, 1)
        X = np.vstack([X, sample_pos])
        y = np.hstack([y, 1])
        X = np.vstack([X, sample_neg])
        y = np.hstack([y, 0])

    clf = MLPClassifier(alpha=1e-5, hidden_layer_sizes=(5, 5), random_state=1)
    clf.fit(X, y)
    predicted = clf.predict(X_test)
    scores1.append(sklearn.metrics.accuracy_score(y_test, predicted))
    
print(np.mean(scores0), np.std(scores0))
print(np.mean(scores1), np.std(scores1))

/home/kevin/anaconda3/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


0.682 0.0628967407741
0.66 0.0464758001545
